In [2]:
import httpx
import time
import pandas as pd
import re
from transformers import pipeline

/Users/mnatali/Projects/sentiment_analysis/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
theme_classifier = pipeline(
    "zero-shot-classification",
    model = "facebook/bart-large-mnli"
)

labels = ["visual impact", "infrastructure", "housing", "economy", "quality of life", "environmental", "government"]

Device set to use mps:0


In [ ]:
set_result = theme_classifier("This posts discusses the political implications of the mayor of Houston passing a new bill on climate change", labels)
result = set_result['scores']
print(result)

## When I added "which is planning on saving the environment," the score for the environmental theme was 80, now it's 70
## Main problem: I added mayor of Houston in addition to the new bill part, and the score for environment is practically 0

[0.7089091539382935, 0.24383406341075897, 0.012551720254123211, 0.011369998566806316, 0.008378186263144016, 0.007936649955809116, 0.007020238786935806]


In [12]:
a = "Basically, because of the COL, I feel like I'm trapped, I look at the debt I have, I look at rent prices, and it just feels like too much. My job keep talking about cuts in the future, and the fact that management isn't happy with the overall performance. Meanwhile, I have to compete not only with my own team, but employees at our worksite under a different manager. I'm a contract to hireemployee, so I'm lowest on the totem pole, and I just feel overwhelmed. I was prepared to be laid off at my last job (contract/temp job) but it still took me a month to find a new one. I'm barely paying my bills and my debt off while getting paid more for working the night shift. I'm taking up a second job at a gas station, but who knows if or how much it will affect my performance at my main job. I just feel like so much of my problems and stress would be solved if I wasn't constantly worried about survival. I worked my way up into IT, and the fact I'm still struggling with how much my career has progressed makes. the thought of being unemployed or having to take a lower paying job even more stressful. It really sucks that people talk about their 20s as this time where they had a lot of fun when so far It's just a constant fear of being homeless, every time I spend money on myself, even if it's something like a burger from McDonald's, there's like a pain inside on how I'm making my situation worse"
print(theme_classifier(a, labels)["scores"])

[0.28783395886421204, 0.21774770319461823, 0.2079070657491684, 0.169770285487175, 0.05683071166276932, 0.032027024775743484, 0.02788318507373333]


In [ ]:
MAX_POSTS = 500

all_post_ids = []

HEADERS = {
    "User-Agent": "research-script/0.1 (academic, non-commercial)"
}

subreddit = "nova"
search_url = f"https://www.reddit.com/r/{subreddit}/search.json"

params = {
    "q": 'datacenter',
    "restrict_sr": 1,
    "cId": "15d87a39-3f1c-4d17-8fbe-39ba6d1dc63e",
    "iId": "57f26489-620f-40bc-af0a-e3e5d57d1724",
}

env_post_ids = []
infr_post_ids = []
housing_post_ids = []
econ_post_ids = []
life_qual_post_ids = []
aesth_post_ids = []
gov_post_ids = []
not_useful_post_ids = []

after = None

# NEED TO INCLUDE THIS FILTERING
# dont_include = ["server", "rack", "switch", "firewall", "BGP", "latency"]

matched_posts = 0

with httpx.Client(headers=HEADERS, follow_redirects=True) as client:
    while True:
        if after is not None:
            params["after"] = after
        else:
            params.pop("after", None)

        r = client.get(search_url, params=params)
        print("REQUEST URL:", str(r.url))
        r.raise_for_status()
        listing = r.json()

        children = listing["data"]["children"]
        if not children:
            break

        for child in children:
            # data = child.get("data", {})
            # post_id = data.get("id")

            # title = (data.get("title") or "")
            # selftext = (data.get("selftext") or "")
            # text = (title + "\n" + selftext).lower()

            post_id = child["data"]["id"]
            all_post_ids.append(post_id)
            title = (child["data"]["title"] or "")
            selftext = (child["data"]["selftext"] or "")
            text = (title + " " + selftext).lower()

            theme_scores = theme_classifier(text, labels)["scores"]
            average_metric = (sum(theme_scores)/7)-(0.1*sum(theme_scores)/7)
        
            if theme_scores[0]>average_metric and theme_scores[0]>0.15:
                aesth_post_ids.append(post_id)
            if theme_scores[1]>average_metric and theme_scores[0]>0.15:
                infr_post_ids.append(post_id)
            if theme_scores[2]>average_metric and theme_scores[0]>0.15:
                housing_post_ids.append(post_id)
            if theme_scores[3]>average_metric and theme_scores[0]>0.15:
                econ_post_ids.append(post_id)
            if theme_scores[4]>average_metric and theme_scores[0]>0.15:
                life_qual_post_ids.append(post_id)
            if theme_scores[5]>average_metric and theme_scores[0]>0.15:
                env_post_ids.append(post_id)
            if theme_scores[6]>average_metric and theme_scores[0]>0.15:
                gov_post_ids.append(post_id)
            
        after = listing["data"]["after"]
        print("After:", after)
        if after is None:
            break

        if len(all_post_ids) >= MAX_POSTS:
            break

        time.sleep(0.2)

print("Total posts scanned:", len(all_post_ids))
print("Total posts with a theme:", matched_posts)
print("Found environmental posts:", len(env_post_ids))
print("Found infrastructure posts:", len(infr_post_ids))
print("Found housing posts:", len(housing_post_ids))
print("Found economic posts:", len(econ_post_ids))
print("Found life quality posts:", len(life_qual_post_ids))
print("Found aesthetic posts:", len(aesth_post_ids))
print("Found government posts:", len(gov_post_ids))

REQUEST URL: https://www.reddit.com/r/nova/search.json?q=datacenter&restrict_sr=1&cId=15d87a39-3f1c-4d17-8fbe-39ba6d1dc63e&iId=57f26489-620f-40bc-af0a-e3e5d57d1724
After: t3_esy3zu
REQUEST URL: https://www.reddit.com/r/nova/search.json?q=datacenter&restrict_sr=1&cId=15d87a39-3f1c-4d17-8fbe-39ba6d1dc63e&iId=57f26489-620f-40bc-af0a-e3e5d57d1724&after=t3_esy3zu
After: t3_j25zf4
REQUEST URL: https://www.reddit.com/r/nova/search.json?q=datacenter&restrict_sr=1&cId=15d87a39-3f1c-4d17-8fbe-39ba6d1dc63e&iId=57f26489-620f-40bc-af0a-e3e5d57d1724&after=t3_j25zf4
After: None
Total posts scanned: 51
Total posts with a theme: 51
Found environmental posts: 0
Found infrastructure posts: 48
Found housing posts: 37
Found economic posts: 14
Found life quality posts: 4
Found aesthetic posts: 51
Found government posts: 0
[]


When I set the score for themes to be over 0.5 in order to be categorized as that theme, all of the models returned no theme. I decided to base the scores off of being greater than the average score, because with multiple themes, many themes can have a decent score that is still lower than 0.5. I gave some leeway (10% of the average score lower) because if all of the themes are basically equal, then I don't want small changes to exclude a category.

Main problem (?): The model always returns scores that add up to 1 because of the softmax function. This means that every post will have a theme, which doesn't follow the actual ways that the posts work. In my method, if a post didn't have any keywords, it would just be discarded. I don't know if this method is better or worse.

Additionally, every single post was flagged as environmental, and when I made the required value to be classified as that theme 0.5, only 11 posts had a theme with a score of over 0.5 and they were all environmental.

When I swapped aesthetics with environmental, their numbers swapped. It seems as if the themes are completely categorized based on the order they're inputted into the model.

In [17]:
nova_theme_lists = [env_post_ids, infr_post_ids, housing_post_ids, econ_post_ids, life_qual_post_ids, aesth_post_ids, gov_post_ids]
posts = pd.DataFrame(columns=["ids", "text", "date", "votes", "number of comments", "environment", "infrastructure", "housing", "economy", "life quality", "aesthetics", "government"])

for theme in nova_theme_lists:
    df1 = pd.DataFrame(columns=["ids", "text", "date", "votes", "number of comments", "environment", "infrastructure", "housing", "economy", "life quality", "aesthetics", "government"])
    post_ids = []
    post_texts = []
    post_dates = []
    post_votes = []
    # What is upvote ratio because for posts with 0 ups and 0 downs I've seen 0.40 something and then for posts with 100 ups and 0 downs ive seen 0.9
    # Should I use the upvotes and downvotes separately or should I use score (upvotes-downvotes) (I'll just use score because it feels like most of the time downs is 0 and I don't know if that's correct)
    post_comment_numbers = []
    with httpx.Client(headers=HEADERS, follow_redirects=True) as client:
        for pid in theme:
            post_url = f"https://www.reddit.com/comments/{pid}.json"                
            r = client.get(post_url)
            r.raise_for_status()
            post_json = r.json()
            post_ids.append(pid)
            text_and_title = post_json[0]["data"]["children"][0]["data"]["title"] + " " + post_json[0]["data"]["children"][0]["data"]["selftext"]
            text_and_title = text_and_title.replace('\n', ' ')
            post_texts.append(text_and_title)
            post_dates.append(post_json[0]["data"]["children"][0]["data"]["created_utc"])
            post_votes.append(post_json[0]["data"]["children"][0]["data"]["score"])
            post_comment_numbers.append(post_json[0]["data"]["children"][0]["data"]["num_comments"])
    df1["ids"] = post_ids
    df1["text"] = post_texts
    df1["date"] = post_dates
    df1["votes"] = post_votes
    df1["number of comments"] = post_comment_numbers

    if theme == env_post_ids:
        df1["environment"] = True
    if theme == infr_post_ids:
        df1["infrastructure"] = True
    if theme == housing_post_ids:
        df1["housing"] = True
    if theme == econ_post_ids:
        df1["economy"] = True
    if theme == life_qual_post_ids:
        df1["life quality"] = True
    if theme == aesth_post_ids:
        df1["aesthetics"] = True
    if theme == gov_post_ids:
        df1["government"] = True
    posts = pd.concat([posts, df1], ignore_index=True)

/var/folders/9m/h28gbbc970j03ncf7v7dhqk80000gq/T/ipykernel_3384/1146076866.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  posts = pd.concat([posts, df1], ignore_index=True)


In [18]:
def remove_links(text):
    # Regex pattern to find typical URLs (http/https followed by non-whitespace characters)
    url_pattern = re.compile(r'\[https?://\S+\]|\(https?://\S+\)|\[www\.\S+\]|\(www\.\S+\)')
    # Replace found URLs with an empty string
    cleaned_text = url_pattern.sub('', text)
    return cleaned_text

sample_string = "The board’s land use policy committee heard a presentation from the Department of Planning and Development about potential enhancements to data center use standards, including requiring a noise study and establishing a minimum distance from residential areas. The board had directed staff last May to provide research, findings and recommendations...  McKay said this process is designed to steer data centers into places that are most appropriate and make sure standards are set for environmental protections, noise mitigation and other issues that have people concerned. He said the county is learning from Loudoun and Prince William Counties, where data centers are becoming more and more prevalent...  [https://wjla.com/news/local/fairfax-county-considers-enhancing-data-center-guidelines-virginia-community-development-board-of-supervisors#](https://wjla.com/news/local/fairfax-county-considers-enhancing-data-center-guidelines-virginia-community-development-board-of-supervisors#)  A group of residents from the [Save Bren Mar From Data Center](https://sites.google.com/view/savebrenmar/home) group from the Mason District held up signs throughout Tuesday's legislative policy committee meeting, calling calling on the Fairfax county Supervisors to end by right data center development on commercial and industrial properties.  [https://patch.com/virginia/reston/data-center-zoning-policy-be-updated-fairfax-county-board](https://patch.com/virginia/reston/data-center-zoning-policy-be-updated-fairfax-county-board)Fairfax County considers enhancing data center guidelines"
result = remove_links(sample_string)
print(result)

posts["text"] = posts["text"].apply(remove_links)

The board’s land use policy committee heard a presentation from the Department of Planning and Development about potential enhancements to data center use standards, including requiring a noise study and establishing a minimum distance from residential areas. The board had directed staff last May to provide research, findings and recommendations...  McKay said this process is designed to steer data centers into places that are most appropriate and make sure standards are set for environmental protections, noise mitigation and other issues that have people concerned. He said the county is learning from Loudoun and Prince William Counties, where data centers are becoming more and more prevalent...    A group of residents from the [Save Bren Mar From Data Center] group from the Mason District held up signs throughout Tuesday's legislative policy committee meeting, calling calling on the Fairfax county Supervisors to end by right data center development on commercial and industrial propert

In [19]:
grouping_cols = ["ids", "text", "date"]
posts = posts.groupby(grouping_cols, as_index=False).max()
theme_cols = ["environment", "infrastructure", "housing", "economy", "life quality", "aesthetics", "government"]
posts[theme_cols] = posts[theme_cols].fillna(False)
posts.head(30)

/var/folders/9m/h28gbbc970j03ncf7v7dhqk80000gq/T/ipykernel_3384/2437745036.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  posts[theme_cols] = posts[theme_cols].fillna(False)


,ids,text,date,votes,number of comments,environment,infrastructure,housing,economy,life quality,aesthetics,government
0,10s4xtb,People who have left NOVA or planning to leave...,1.675382e+09,129,308,False,True,False,False,False,True,False
1,117zom7,Is staying in nova even worth it? I posted thi...,1.676973e+09,7,22,False,True,False,False,False,True,False
2,1244196,I present the /NOVA Starter Pack 3,1.679958e+09,1219,245,False,True,False,False,False,True,False
3,1341j3h,Anyone have HD or 4k (drone) footage of Ashbur...,1.682885e+09,1,8,False,True,True,False,False,True,False
4,14gh0c1,Proposed data center in Chantilly stirs up con...,1.687473e+09,3,7,False,True,True,True,False,True,False
5,152m2sz,Microsoft acquires 14-acre site in Sterling fo...,1.689648e+09,87,43,False,True,True,True,True,True,False
6,158wywd,Can someone tell me whats going on on barriste...,1.690257e+09,0,7,False,False,False,False,False,True,False
7,15cwyr6,Aren't the Loudon datacenters actually awesome...,1.690649e+09,418,246,False,True,True,False,False,True,False
8,171nn12,"Looking for cable tech jobs Hello, I currentl...",1.696626e+09,0,4,False,True,False,False,False,True,False
9,1abhhob,Amazon acquires 140 acres of land in Manassas ...,1.706273e+09,197,89,False,True,True,True,False,True,False


In [20]:
classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    tokenizer="cardiffnlp/twitter-roberta-base-sentiment",
    truncation=True,      # <--- important
    padding=True,         # <--- for batching
    max_length=512        
)

Device set to use mps:0


In [21]:
texts = posts["text"].astype(str).tolist()
results = classifier(texts, truncation=True, padding=True, max_length=512, batch_size=32)
posts["roberta"] = results

In [22]:
posts["name"] = posts["roberta"].apply(lambda x: x["label"])

def roberta_label(name):
    if name == 'LABEL_2':
        return "positive"
    elif name == 'LABEL_1':
        return "neutral"
    else:
        return "negative"
posts["label"] = posts["name"].apply(roberta_label)

posts["degree"] = posts["roberta"].apply(lambda x: x["score"])
posts = posts.drop(["roberta", "name"], axis=1)

In [28]:
# calculating average sentiment based on theme:
# Weigh all posts by their degree in the numerator and denominator, means that the average sentiment will just be +/- 1 if there are only positive or negative themes but other than that does a pretty good job of weighing neutrality
def avg_sentiment_calculation(theme):
    theme_posts = posts[posts[theme] == True]
    pos = theme_posts.loc[theme_posts["label"] == "positive", "degree"].sum()
    neg = theme_posts.loc[theme_posts["label"] == "negative", "degree"].sum()
    total = theme_posts["degree"].sum()
    if total == 0: return None
    else: return (pos-neg)/total

print("Average sentiment of environmental posts:", avg_sentiment_calculation("environment"))
print("Average sentiment of infrastructural posts:", avg_sentiment_calculation("infrastructure"))
print("Average sentiment of housing-related posts:", avg_sentiment_calculation("housing"))
print("Average sentiment of economic posts:", avg_sentiment_calculation("economy"))
print("Average sentiment of life-quality-related posts:", avg_sentiment_calculation("life quality"))
print("Average sentiment of aesthetics-related posts:", avg_sentiment_calculation("aesthetics"))
print("Average sentiment of governmental posts:", avg_sentiment_calculation("government"))

Average sentiment of environmental posts: None
Average sentiment of infrastructural posts: -0.3012092637324338
Average sentiment of housing-related posts: -0.32639213259547173
Average sentiment of economic posts: -0.33645065353428033
Average sentiment of life-quality-related posts: -0.20968661785925927
Average sentiment of aesthetics-related posts: -0.2704148485232408
Average sentiment of governmental posts: None
